# Distributed training with Keras

## Overview

The tf.distribute.Strategy API provides an abstraction for distributing your training across multiple processing units. The goal is to allow users to enable distributed training using existing models and training code, with minimal changes.

This tutorial uses the tf.distribute.MirroredStrategy, which does in-graph replication with synchronous training on many GPUs on one machine. Essentially, it copies all of the model's variables to each processor. Then, it uses all-reduce to combine the gradients from all processors and applies the combined value to all copies of the model.

MirroredStrategy is one of several distribution strategy available in TensorFlow core. You can read about more strategies at distribution strategy guide.

### Keras API

This example uses the tf.keras API to build the model and training loop. For custom training loops, see the tf.distribute.Strategy with training loops tutorial.

## Import dependencies

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Import TensorFlow and TensorFlow Datasets
try:
  !pip install -q tf-nightly
except Exception:
  pass

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
print(tf.__version__)

2.2.0-dev20200203


## Download the dataset

Download the MNIST dataset and load it from TensorFlow Datasets. This returns a dataset in tf.data format.

Setting with_info to True includes the metadata for the entire dataset, which is being saved here to info. Among other things, this metadata object includes the number of train and test examples.

In [3]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

C:\ProgramData\Anaconda3\envs\spyder\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\ProgramData\Anaconda3\envs\spyder\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\ProgramData\Anaconda3\envs\spyder\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset mnist downloaded and prepared to C:\Users\BlackWS\tensorflow_datasets\mnist\1.0.0. Subsequent calls will reuse this data.


## Define distribution strategy

Create a MirroredStrategy object. This will handle distribution, and provides a context manager (tf.distribute.MirroredStrategy.scope) to build your model inside.

In [4]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [5]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


## Setup input pipeline

When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. In general, use the largest batch size that fits the GPU memory, and tune the learning rate accordingly.

In [6]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

Pixel values, which are 0-255, have to be normalized to the 0-1 range. Define this scale in a function.

In [7]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

Apply this function to the training and test data, shuffle the training data, and batch it for training. Notice we are also keeping an in-memory cache of the training data to improve performance.

In [8]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

## Create the model

Create and compile the Keras model in the context of strategy.scope.

In [9]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

## Define the callbacks

The callbacks used here are:

* TensorBoard: This callback writes a log for TensorBoard which allows you to visualize the graphs.
* Model Checkpoint: This callback saves the model after every epoch.
* Learning Rate Scheduler: Using this callback, you can schedule the learning rate to change after every epoch/batch.

For illustrative purposes, add a print callback to display the learning rate in the notebook.

In [10]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [11]:
# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [12]:
# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [13]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

## Train and evaluate

Now, train the model in the usual way, calling fit on the model and passing in the dataset created at the beginning of the tutorial. This step is the same whether you are distributing the training or not.

In [14]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Epoch 1/12
    938/Unknown - 15s 17ms/step - loss: 0.1974 - accuracy: 0.9449
Learning rate for epoch 1 is 0.0010000000474974513
938/938 [==============================] - 16s 17ms/step - loss: 0.1974 - accuracy: 0.9449
Epoch 2/12
933/938 [============================>.] - ETA: 0s - loss: 0.0669 - accuracy: 0.9807
Learning rate for epoch 2 is 0.0010000000474974513
938/938 [==============================] - 11s 11ms/step - loss: 0.0670 - accuracy: 0.9807
Epoch 3/12
936/938 [============================>.] - ETA: 0s - loss: 0.0472 - accuracy: 0.9856
Learning rate for epoch 3 is 0.0010000000474974513
938/938 [==============================] - 11s 12ms/step - loss: 0.0472 - accuracy: 0.9856
Epoch 4/12
935/938 [============================>.] - ETA: 0s - loss: 0.0262 - accuracy: 0.9928
Learning rate for epoch 4 is 9.999999747378752e-05
938/938 [==============================] - 11s 12ms/step - loss: 0.0262 - accuracy: 0.9928
Epoch 5/12
933/938 [============================>.] - ETA: 0s - los

As you can see below, the checkpoints are getting saved.

In [16]:
# check the checkpoint directory
!ls {checkpoint_dir}

checkpoint
ckpt_1.data-00000-of-00001
ckpt_1.index
ckpt_10.data-00000-of-00001
ckpt_10.index
ckpt_11.data-00000-of-00001
ckpt_11.index
ckpt_12.data-00000-of-00001
ckpt_12.index
ckpt_2.data-00000-of-00001
ckpt_2.index
ckpt_3.data-00000-of-00001
ckpt_3.index
ckpt_4.data-00000-of-00001
ckpt_4.index
ckpt_5.data-00000-of-00001
ckpt_5.index
ckpt_6.data-00000-of-00001
ckpt_6.index
ckpt_7.data-00000-of-00001
ckpt_7.index
ckpt_8.data-00000-of-00001
ckpt_8.index
ckpt_9.data-00000-of-00001
ckpt_9.index


To see how the model perform, load the latest checkpoint and call evaluate on the test data.

Call evaluate as before using appropriate datasets.

In [17]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

    157/Unknown - 2s 14ms/step - loss: 0.0387 - accuracy: 0.9866Eval loss: 0.038743957203878146, Eval Accuracy: 0.9865999817848206


To see the output, you can download and view the TensorBoard logs at the terminal.

In [20]:
!ls -sh ./logs

total 4.0K
4.0K train


## Export to SavedModel

Export the graph and the variables to the platform-agnostic SavedModel format. After your model is saved, you can load it with or without the scope.

In [21]:
path = 'saved_model/'

In [22]:
model.save(path, save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


Load the model without strategy.scope.

In [23]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

    157/Unknown - 1s 8ms/step - loss: 0.0387 - accuracy: 0.9866Eval loss: 0.038743957203878146, Eval Accuracy: 0.9865999817848206


Load the model with strategy.scope.

In [24]:
with strategy.scope():
  replicated_model = tf.keras.models.load_model(path)
  replicated_model.compile(loss='sparse_categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

  eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
  print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

    157/Unknown - 2s 13ms/step - loss: 0.0387 - accuracy: 0.9866Eval loss: 0.038743957203878146, Eval Accuracy: 0.9865999817848206


## Examples and Tutorials

Here are some examples for using distribution strategy with keras fit/compile:

* Transformer example trained using tf.distribute.MirroredStrategy
* NCF example trained using tf.distribute.MirroredStrategy.

More examples listed in the Distribution strategy guide

## Next steps

* Read the distribution strategy guide.
* Read the Distributed Training with Custom Training Loops tutorial.